# *Versions*
- 1.0  
-- Primeiro teste  
- 1.1  
-- Trocado copy_rates_range para copy_rates_from_pos por conta da perda de dados  
-- Passando funções para robot_functions.py  
-- Primeira implementação de uma GUI  
- 1.2  
-- Colocado todos EUR  
-- Resolvido o bug 0001  
- 1.3
-- Importado ipynb para py  
-- Trocado o tempo do sleep de 0.5s para 0.1s  
-- Colocado restante dos pares  
- 1.4  
-- Removido botão exit, apenas sairá apertando X. Motivo: Irrelevante para já  
-- Janela é aberta em um tamanho de 960x320 e é possível ajusta-la  
-- Colocado balance, margin free e profit no topo, com cores e background  
- 1.5  
-- Resolvido bug 00002
-- Primeiro teste na AWS, houve problemas com a biblioteca myplotlib   
-- Printa a que preço foi comprado/vendido  
- 1.6  
-- Criado o get_data para pegar os dados, jogado no modulo
-- Passado risco como constante no módulo robot_functions.py  
-- Removido matplotlib e numpy  
-- Versão no topo da GUI é uma constante  
- 1.7  
-- Removido import do datetime, motivo: não estava sendo usado  
-- Dividido robot_functions em 3 outros modulos, mt5, financial e data  
-- Criado um novo modulo para lidar com a GUI  
-- Removido from IPython.display import clear_output, motivo: não era usado
-- Passado bestime e magic number como constantes  
- 1.8  
-- Ao inicializar, irá checar se o algotrading está ativado, caso não, desliga a ligação  
-- Colocado ping e estado da conexão na gui  
- 1.9  
-- Removido GUI para maior otimização
-- Removido account_info() do robo e passa para o proprio modulo, reduzido 2 parametros para abertura de ordem  
-- Ajustado erro grave em flags faltantes  
-- Resolvido bug 00003 e bug 00004  
-- Reduzido todos os if elif para uma lista de dicionarios  
- 2.0  
-- Resolvido bug 00005  
-- Agora em vez de lista, os eventos são apenas dicionarios  
-- Arrumado erro no login do mt5, apenas logava em uma conta em vez da conta passada coo parametro  
-- Mudada estrategia, em vez de besttime agora o valor tem que estar dentro de 2 desvios padroes
-- Robo agora opera por hora, no indice -2
-- Mudado o deviation de 0 para 10. Evitar erro 10004


# *Bugs*  
### Abertos  

### Resolvidos  
- Bug 00001:  
-- Problema: Quando é feito uma operação bem sucedidade, uma mensagem de erro aparece. Esta retornando algum outro codigo.  
-- Solução: Estava retornando uma lista, apenas coloquei no if o result[0] para pegar o código 10009
-- Implementação: Feito na versão 1.2  
- Bug 00002:
-- Problema: Nome da função de fechar esta errada em tudo e excluir certos parametros  
-- Solução: Ajustei o nome removendo a letra a mais, como também tirei o parametro obrigatorio do magic number, ja não é mais necessario  
-- Implementação: Feito na versão 1.5  
- Bug 00003?:  
-- Possivel erro 10014, volume invalido?  
-- Solução: NZDCAD, NZDCHF e NZDJPY possuem um volume minimo de 0.1. Pelo menos na conta do MetaQuotes  
-- Implementação: Feito na versão 1.9  
- Bug 00004:  
-- Problema: Ping abaixo de 100ms continua amarelo  
-- Solução: Removido GUI  
-- Implementação: Feito na versão 1.9  
- Bug 00005:  
-- Problema: Erro 10004, nova cotação  
-- Solução: Colocado um elif diferenciado para saber diferenciar, apesar que não havia risco  
-- Implementação: Feito na versão 2.0  

# *Futuras Atualizações*  
-> Por o comentario da comrpa/venda no programa   
-> Possivel integração com o Telegram   
-> Desenho tanto do capital como variação dos numeros  
-> Mostrar operações ativas, assim como outras informações  
-> Importar py para um exe   
-> Manter os textso da gui em um lugar fixo conforme a tela aumenta
-> Retira GUI para ja, possivel diminuição do uso de CPU pela metade com um sleep de 0.1

# *Tarefas*
-> Entender o resultado do account_info  
-> O indice 0 é o mais antigo, o ultimo é o mais novo

In [1]:
import MetaTrader5 as mt5
import pandas as pd
from time import sleep
import financial as fn
from data import get_data
from mt5 import login_mt5
from time import sleep

In [2]:
# Constantes

HONEMAGICNUMBER = 1010
EVENTS = fn.MY_EVENTS

In [3]:
login_mt5(login=41442678,password='0cxebqvs',server='MetaQuotes-Demo')

while True:
    
    data = get_data(start=0,end=60_000)

    for i in EVENTS.values():
        if 0 > data[i['first_currency']].iloc[-2] > -(data[i['first_currency']].std()*2) and 0 < data[i['second_currency']].iloc[-2] < (data[i['second_currency']].std()*2) and  i['buy']:
            if not i['sell']:
                fn.close_trade('buy',i['sell_request'])
                i['buy'], i['sell'], i['buy_request'] = fn.open_trade(action='buy', symbol=i['symbol'], ea_magic_number=HONEMAGICNUMBER)
            else:
                i['buy'], _,i['buy_request'] = fn.open_trade(action='buy', symbol=i['symbol'], ea_magic_number=HONEMAGICNUMBER)
        elif 0 < data[i['first_currency']].iloc[-2] < (data[i['first_currency']].std()*2) and 0 > data[i['second_currency']].iloc[-2] > -(data[i['second_currency']].std()*2) and i['sell']:
            if not i['buy']:
                fn.close_trade('sell',i['buy_request'])
                i['sell'], i['buy'], i['sell_request'] = fn.open_trade(action='sell', symbol=i['symbol'], ea_magic_number=HONEMAGICNUMBER)
            else:
                i['sell'], _, i['sell_request'] = fn.open_trade(action='sell', symbol=i['symbol'], ea_magic_number=HONEMAGICNUMBER)

    sleep(1800)
                
mt5.shutdown()

Sell | Symbol: EURCHF | Price: 1.08059 | Volume: 0.05
Sell | Symbol: EURGBP | Price: 0.86911 | Volume: 0.05
Error Order GBPAUD: (1, 'Success') | Code: 10004 | New Price.
Buy | Symbol: GBPCAD | Price: 1.76384 | Volume: 0.05
Buy | Symbol: GBPUSD | Price: 1.3854600000000001 | Volume: 0.05
Buy | Symbol: GBPNZD | Price: 1.9328400000000001 | Volume: 0.05
Sell | Symbol: USDCHF | Price: 0.89738 | Volume: 0.05
Sell | Symbol: AUDCHF | Price: 0.69441 | Volume: 0.05
Sell | Symbol: CADCHF | Price: 0.70483 | Volume: 0.05
Sell | Symbol: NZDCHF | Price: 0.6433 | Volume: 0.1


True

In [10]:
events = EVENTS
events['EURCHF']['sell_request']['symbol']

'EURCHF'

In [11]:
login_mt5(login=41442678,password='0cxebqvs',server='MetaQuotes-Demo')
position_id=mt5.positions_get(symbol=events['EURCHF']['sell_request']['symbol'])
mt5.shutdown()

True

In [14]:
position_id[0][0]

870533238

In [15]:
position_id

(TradePosition(ticket=870533238, time=1613581185, time_msc=1613581185125, time_update=1613581185, time_update_msc=1613581185125, type=1, magic=1010, identifier=870533238, reason=3, volume=0.05, price_open=1.08059, sl=0.0, tp=0.0, price_current=1.08071, swap=0.0, profit=-0.56, symbol='EURCHF', comment='MAYBE LATER A CO', external_id=''),)